🛠️ How to run (with your config)

You trained with:

window: 64
horizon: 12
model: tptrans (d_model=192, nhead=4, enc=4, dec=2)


So evaluate with the same horizon=12:

A) Batch all (per-MMSI folders, iterative rollout)
python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 90 \
  --mmsi all \
  --out_dir data/figures/test3 \
  --auto_extent \
  --extent_outlier_sigma 3.0 \
  --debug_save_npz

B) Only a few MMSIs
python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 75 \
  --mmsi 210046000,210174000 \
  --out_dir data/figures/test_subset2 \
  --auto_extent

C) Exactly one trip (MMSI + trip_id)
python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 85 \
  --mmsi 219000111 \
  --trip_id 0 \
  --out_dir data/figures/one_trip


Tip: omit --auto_extent to keep the map locked to Denmark for all figures.

python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 90 \
  --mmsi all \
  --out_dir data/figures/test01 \
  --auto_extent \
  --extent_outlier_sigma 3.0 \
  --debug_save_npz


python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 2 \
  --pred_cut 75 \
  --mmsi 210046000,210174000 \
  --out_dir data/figures/test_subset01 \
  --auto_extent






Arg cheatsheet (with exactly what to use)

--split_dir
Where your *_processed.pkl trips live (e.g., data/map_reduced/val).

--ckpt
Checkpoint file (e.g., data/checkpoints/traj_tptrans.pt).

--model
tptrans (your trained model) or gru baselines.

--horizon
Use 12 to match training. It’s the chunk size per rollout step.

--past_len
How many past steps to feed per rollout (≤ training window, i.e., 64).

--pred_cut
Percent of the full trip used as past.
75 → blue 75% / green 25% (and red now distance-matched to green).
90 → blue 90% / green 10%.

--cap_future (optional)
Hard cap on future steps if you don’t want to use the whole tail.

--mmsi
all or a comma-separated list like 210046000,210174000.
For a single specific trip: add --trip_id 0.

--out_dir
Root output folder. The script creates per-MMSI subfolders.

--auto_extent
Zoom to track (clamped to DK). Omit it to lock to Denmark bounds.

--debug_save_npz
Save all arrays for inspection.

Everything else can stay at defaults.

Re-run your two examples

A) All MMSIs, consistent with training:

python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 90 \
  --mmsi all \
  --out_dir data/figures/test01 \
  --auto_extent \
  --extent_outlier_sigma 3.0 \
  --debug_save_npz


B) Specific MMSIs (keep --horizon 12!):

python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 75 \
  --mmsi 210046000,210174000 \
  --out_dir data/figures/test_subset01 \
  --auto_extent

Outputs (per MMSI folder)

Plot: traj_tptrans_mmsi-<MMSI>_trip-<id>_cut-<pct>_idx-<i>.png

CSV of what’s plotted (past/true/pred + timestamps): trip_<MMSI>_<id>_cut-<pct>_idx-<i>.csv

Metrics appended per trip: metrics_<MMSI>.csv (ADE/FDE/median-AE in km)

Optional NPZ dump: debug_npz/debug_<MMSI>_<id>_idx-<i>.npz

Arg quick guide (use these)

--split_dir: where your *_processed.pkl are (e.g., data/map_reduced/val).

--ckpt: your checkpoint (e.g., data/checkpoints/traj_tptrans.pt).

--model: tptrans (yours) or gru.

--horizon: 12 (match training).

--past_len: 64 (≤ training window).

--pred_cut: percent of points used as past.

75 → 75% blue / 25% green.

90 → 90% blue / 10% green.
(We now make the red path’s km match the green path’s km.)

--cap_future (optional): cap the green/red tail to N points if you don’t want the whole tail.

--mmsi: all or comma-separated list like 210046000,210174000. For one exact trip: add --trip_id 0.

--out_dir: output root (plots, per-MMSI folders, CSVs, metrics).

--auto_extent: zoom to the track (clamped to DK). Omit to lock to Denmark.

--min_points: raise this (e.g., 60) to avoid micro-trips.





Recommended commands for your case

Subset, robust to short trips, consistent with training:

python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 75 \
  --mmsi 210046000,210174000 \
  --min_points 60 \
  --out_dir data/figures/test_subset04 \
  --auto_extent


All MMSIs with 90% cut (and short-trip filter):

python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 90 \
  --min_points 60 \
  --mmsi all \
  --out_dir data/figures/test01 \
  --auto_extent \
  --extent_outlier_sigma 3.0 \
  --debug_save_npz

How to run (matching your training)

horizon = 12 (your model’s prediction head)

past_len = 64 (your training window)

pred_cut is % of points used as past (blue); the rest is true future (green).
The red is now trimmed to the same km as green.

Examples:

# Subset (two MMSIs), with auto-zoom
python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 75 \
  --min_points 60 \
  --mmsi 210046000,210174000 \
  --out_dir data/figures/test_subset04 \
  --auto_extent \
  --verbose

# All MMSIs, 90% cut, save debug arrays
python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 90 \
  --min_points 60 \
  --mmsi all \
  --out_dir data/figures/test01 \
  --auto_extent \
  --extent_outlier_sigma 3.0 \
  --debug_save_npz \
  --verbose

Quick arg glossary

--horizon — model’s per-pass forecast length (use 12 to match training).

--past_len — past context fed each rollout (≤ window; use 64).

--pred_cut — % of points for past (blue). (75→ 75% blue / 25% green).

--cap_future — optional maximum future points to consider.

--min_points — skip trips shorter than this (before splitting).

--mmsi — all, or comma-separated MMSIs; add --trip_id to pick an exact trip.

--auto_extent — zoom to the trajectory; omit to lock DK extent.

--out_dir — output root; figures, trip CSV, and metrics CSV are stored under out_dir/<MMSI>/

--match_distance

python -m src.eval.eval_traj_newnewnew \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 75 \
  --min_points 60 \
  --mmsi 210046000,210174000 \
  --out_dir data/figures/test_subset07 \
  --auto_extent \
  --verbose \
  --match_distance

How to run
1) Evaluate all MMSIs in the split

Plots, CSVs, and metrics will be written under --out_dir/<MMSI>/...

python -m src.eval.eval_traj \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 90 \
  --min_points 60 \
  --mmsi all \
  --out_dir data/figures/val_all_cut90 \
  --auto_extent \
  --extent_outlier_sigma 3.0


What you’ll see per MMSI folder

traj_tptrans_mmsi-<MMSI>_trip-<id>_cut-<pct>_idx-<i>.png — plot

Blue = first pred_cut% (past)

Black dot = last blue point

Green = full true future (cut → end of trip)

Red = predicted future aligned to the first N_future timestamps; if --match_distance is set, it’s trimmed only if longer in km

trip_<MMSI>_<id>_cut-<pct>_idx-<i>.csv — the series that was plotted:

All past points, full true future, and the aligned predicted future with timestamps

metrics_<MMSI>.csv — append-only metrics per trip:

ADE (km), FDE (km), median AE (km), plus counts used

2) Evaluate a subset of MMSIs
python -m src.eval.eval_traj \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 75 \
  --min_points 60 \
  --mmsi 210046000,210174000 \
  --out_dir data/figures/val_subset_cut75 \
  --auto_extent

3) Evaluate a single specific trip
python -m src.eval.eval_traj \
  --split_dir data/map_reduced/val \
  --ckpt data/checkpoints/traj_tptrans.pt \
  --model tptrans \
  --horizon 12 \
  --past_len 64 \
  --pred_cut 75 \
  --min_points 30 \
  --mmsi 210046000 \
  --trip_id 0 \
  --out_dir data/figures/val_single_cut75 \
  --auto_extent \
  --verbose

A few notes so results are consistent

horizon (12) must match the checkpoint’s head.

past_len (64) should be ≤ your training window.

pred_cut is % of points, not distance/time. The plot always shows the full green tail; metrics are computed on the first N_future points (or --cap_future, if set), and the red is aligned to those timestamps.

match_distance only trims if red is longer in km than green. If red is shorter, we leave it (prevents collapse).

auto_extent zooms but stays within DK; omit to lock to [6E–16E, 54N–58N]